# Install Lib

In [2]:
!pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


# Main code

In [79]:
%%writefile create_web.py

import streamlit as st
import cv2
import numpy as np
from PIL import Image
import io
import base64
st.set_page_config(layout="wide")

def get_image_download_link(image, filename='download.jpeg', text='Download Image', original_size=None):
    """Generates a link to download a particular image file."""
    if isinstance(image, np.ndarray):
        img = Image.fromarray(image.astype(np.uint8))
    elif isinstance(image, Image.Image):
        img = image
    else:
        raise TypeError("Unsupported image type. `image` must be either a numpy array or PIL.Image.")
    if original_size:
        img = img.resize(original_size, Image.LANCZOS)
    buffered = io.BytesIO()
    img.save(buffered, format='JPEG')
    img_str = base64.b64encode(buffered.getvalue()).decode()
    href = f'<a href="data:image/jpeg;base64,{img_str}" download="{filename}">{text}</a>'
    return href

def read_image(upload_file):
    image_pil = Image.open(upload_file)
    image_np = np.array(image_pil.convert('RGB'))
    return image_np, image_pil

def color_balance(image, red_factor, green_factor, blue_factor):
    b, g, r = cv2.split(image)
    #scale to 0-2.55
    red_channel = cv2.multiply(r, red_factor/100)
    green_channel = cv2.multiply(g, green_factor/100)
    blue_channel = cv2.multiply(b, blue_factor/100)
    balanced_image = cv2.merge((red_channel, green_channel, blue_channel ))
    return balanced_image

def histogram_equalization(image):
    if len(image.shape) == 2:  # Grayscale image
        equalized_img = cv2.equalizeHist(image)
    else:
        b, g, r = cv2.split(image)
        b_eq = cv2.equalizeHist(b)
        g_eq = cv2.equalizeHist(g)
        r_eq = cv2.equalizeHist(r)
        equalized_img = cv2.merge((r_eq, g_eq, b_eq ))
    return equalized_img

def wrappingImage(img, kernelSize : int):
    ### width of the mask
    w = kernelSize // 2

    # Fetch First Rows of for wrapping
    fetchFirstRows = img[0 : w , :]
    fetchLastRows = img[-w : , : ]

    imgWrapped = img.copy()
    imgWrapped = np.insert(imgWrapped, 0, fetchLastRows, axis=0)
    imgWrapped = np.append(imgWrapped, fetchFirstRows, axis = 0)

    ### Fetch First and Last Colsfro wrapping
    fetchFirstCols = imgWrapped[:, 0 : w]
    fetchLastCols = imgWrapped[:, -w : ]
    imgWrapped = np.concatenate([fetchLastCols,imgWrapped], axis=1)
    imgWrapped = np.append(imgWrapped, fetchFirstCols, axis = 1)

    return imgWrapped

def medianFilter(orginalImg, kernelSize : int):

    filteredImage = np.zeros(orginalImg.shape,dtype=np.int32)
    image_h, image_w = orginalImg.shape[0], orginalImg.shape[1]
    w = kernelSize//2
    wrappedImg = wrappingImage(orginalImg, kernelSize)

    for i in range(w, image_h - w): ## traverse image row
        for j in range(w, image_w - w):  ## traverse image col
            overlapImg = wrappedImg[i-w : i+w+1, j-w : j+w+1 ]    # Crop image for mask product
            filteredImage[i][j] = np.median(overlapImg.reshape(-1, 3), axis=0)  # Filtering

    return filteredImage
    # return cv2.medianBlur(img, kernel)

def meanFilter(orginalImg, kernelSize : int):
    filteredImage = np.zeros(orginalImg.shape,dtype=np.int32)
    image_h, image_w = orginalImg.shape[0], orginalImg.shape[1]
    w = kernelSize//2
    wrappedImg = medianFilter(orginalImg, kernelSize)
    for i in range(w, image_h - w): ## traverse image row
        for j in range(w, image_w - w):  ## traverse image col
            total = [0,0,0]
            for m in range(kernelSize):
                for n in range(kernelSize):
                    total += wrappedImg[i-w+m][j-w+n]
            filteredImage[i-w][j-w] = total // (kernelSize * kernelSize)
    return filteredImage

def gaussion_smoothing(image_np, kernel):
  original_image = image_np
  gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
  kernel_size = (kernel, kernel)
  sigma = 1.5
  filtered_image = cv2.GaussianBlur(gray_image, kernel_size, sigma)
  return filtered_image

def edit_image():
    image_upload = st.file_uploader("Upload a photo")
    if image_upload is None:
        st.warning("Please upload an image.")
        st.stop()

    image_np, _ = read_image(image_upload)
    task_options = ('Color Balance', 'Histogram Equalization', 'Median Filter', 'Mean Filter', 'Gaussian Smoothing')
    current_task = st.sidebar.radio("Choose task:", task_options)
    _, center, __ = st.columns(3)

    if current_task == 'Color Balance':
        red_factor = st.sidebar.slider("Red factor", 0, 255, 100)
        green_factor = st.sidebar.slider("Green factor", 0, 255, 100)
        blue_factor = st.sidebar.slider("Blue factor", 0, 255, 100)
        with center:
          st.subheader("Transform image")
          with st.spinner('Generating...'):
            edited_image = color_balance(image_np, red_factor, green_factor, blue_factor)
            st.image(edited_image, use_column_width=True, caption="Edited Image")
          st.success('Done!')
          link = get_image_download_link(edited_image)
          st.subheader('Click on link below to download image')
          st.markdown(link, unsafe_allow_html=True)

    elif current_task == 'Histogram Equalization':
      with center:
        st.subheader("original image")
        st.image(image_np, use_column_width=True)
        submitted = st.button("Apply Histogram Equalization")
        if submitted:
          st.subheader("Transform image")
          with st.spinner('Generating...'):
            edited_image = histogram_equalization(image_np)
            st.image(edited_image, use_column_width=True, caption="Edited Image")
          st.success('Done!')
          link = get_image_download_link(edited_image)
          st.subheader('Click on link below to download image')
          st.markdown(link, unsafe_allow_html=True)

    elif current_task == 'Median Filter':
      with center:
        st.subheader("original image")
        st.image(image_np, use_column_width=True)
        kernel = st.number_input("Choose number of kernel: ", min_value=1, step=2)
        if int(kernel) % 2 == 0:
          st.warning("kernel_size must be an odd number.")
          st.stop()
        if kernel:
          st.subheader("Transform image")
          with st.spinner('Generating...'):
            edited_image = meanFilter(image_np, kernelSize=int(kernel))
            st.image(edited_image, use_column_width=True, caption="Edited Image")
          st.success('Done!')
          link = get_image_download_link(edited_image)
          st.subheader('Click on link below to download image')
          st.markdown(link, unsafe_allow_html=True)

    elif current_task == 'Mean Filter':
      with center:
        st.subheader("original image")
        st.image(image_np, use_column_width=True)
        kernel = st.number_input("Choose number of kernal (normaly is 3): ", min_value=1, step=1)
        if kernel:
          st.subheader("Transform image")
          with st.spinner('Generating...'):
            edited_image = medianFilter(image_np, kernelSize=int(kernel))
            st.image(edited_image, use_column_width=True, caption="Edited Image")
          st.success('Done!')
          link = get_image_download_link(edited_image)
          st.subheader('Click on link below to download image')
          st.markdown(link, unsafe_allow_html=True)

    elif current_task == 'Gaussian Smoothing':
      with center:
        st.subheader("original image")
        st.image(image_np, use_column_width=True)
        kernel = st.number_input("Choose number of kernel: ", min_value=1, step=2)
        if int(kernel) % 2 == 0:
          st.warning("kernel_size must be an odd number.")
          st.stop()
        if kernel:
          st.subheader("Transform image")
          with st.spinner('Generating...'):
            edited_image = gaussion_smoothing(image_np, kernel=int(kernel))
            st.image(edited_image, use_column_width=True, caption="Edited Image")
          st.success('Done!')
          link = get_image_download_link(edited_image)
          st.subheader('Click on link below to download image')
          st.markdown(link, unsafe_allow_html=True)


if __name__ == "__main__":
    edit_image()


Overwriting create_web.py


# Runing webapp

In [81]:
import subprocess
import sys

def get_public_ip(service_url="ipv4.icanhazip.com"):
    try:
        ip_output = subprocess.run(["curl", service_url], capture_output=True, text=True, check=True)
        return ip_output.stdout.strip()
    except subprocess.CalledProcessError as e:
        print(f"Failed to get IP: {e}", file=sys.stderr)
        return None

def run_streamlit():
    try:
        return subprocess.Popen(["streamlit", "run", "create_web.py", "--server.enableXsrfProtection", "false"])
    except Exception as e:
        print(f"Failed to start Streamlit: {e}", file=sys.stderr)
        return None

def run_local_tunnel():
    try:
        proc = subprocess.Popen(["npx", "localtunnel", "--port", "8501"], stdout=subprocess.PIPE, text=True)
        return proc.stdout.readline().strip()
    except Exception as e:
        print(f"Failed to start localtunnel: {e}", file=sys.stderr)
        return None

if __name__ == "__main__":
    public_ip = get_public_ip()
    if public_ip:
        print(f"Public IP address (use as Tunnel Password): \n{public_ip}")

    streamlit_process = run_streamlit()
    if streamlit_process:
        print("Streamlit running...")

    tunnel_url = run_local_tunnel()
    if tunnel_url:
        print(f"Access web app at: {tunnel_url}")


Public IP address (use as Tunnel Password): 
34.125.236.205
Streamlit running...
Access web app at: your url is: https://bright-bats-run.loca.lt
